In [2]:
# 11 news categories
news_links =[ 'big-data', 'cloud']
news_categories=['AI','Cloud']
base_url = 'https://www.ithome.com.tw/'

In [3]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time

In [4]:
%%time
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []

Wall time: 0 ns


In [5]:
for i, url_short_name in enumerate(news_links):  #針對每一類 共有11類

    category = news_categories[i]

    # Categorical url link
    category_url = base_url + url_short_name
    print("Getting categorical news:", category)
    print("Getting categorical news url:", category_url)

    # Request the categorical news page
    req = requests.get(category_url)
    page = BeautifulSoup(req.text, 'lxml')

    items = page.findAll('div', {'class': re.compile("view-content")})[2].findAll('div', {'class': re.compile("span4 channel-item")})

    serial_no = 1
    for j, item in enumerate(items): #針對每一篇項目 抓其細節
        # title
        title = item.find('p',{'class':"title"}).text
        print(serial_no,'--', title )
        
        # news link
        link = item.find('p',{'class':"title"}).find('a')['href']
        link = base_url + link
        # print(link)

        # photo link
        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
            # print(photo_link)
        except:
            photo_link = ''

        # date
        # get date and convert it with the format of "2022-04-20"
        news_date_str = item.find('p', {'class':"post-at"}).text.strip()
        # print(news_date_str)
        # convert the string to a datetime object
        dtime = datetime.strptime(news_date_str, "%Y-%m-%d")
        # convert to the format of "20220420"
        dtstr = dtime.strftime("%Y%m%d")


        page = BeautifulSoup(requests.get( link ).text,'lxml')
        # Find content
        content = page.find('div',{'class':"field-type-text-with-summary"})
        
        try:
            content.find('span').decompose()
        except:
            pass

        cont = content.text

        contents.append(cont)
        categories.append(category)
        titles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        dates.append(news_date_str)      
        item_id.append(url_short_name + "_" + dtstr + "_" + str(serial_no))
        serial_no += 1
        
        time.sleep(2)  # 遵守爬蟲禮節，請小睡一下，以防被擋

        if (j==9): # Here we crawl only 5 pieces of news.
            break

Getting categorical news: AI
Getting categorical news url: https://www.ithome.com.tw/big-data
1 -- 微軟傳已投入開發自有AI專用晶片 
2 -- Adobe Lightroom新增以AI消除噪點、強化細節修整的功能 
3 -- Snapchat開放聊天機器人供全球用戶免費體驗，推出AI生成的濾鏡 
4 -- 馬斯克要發展AI聊天機器人TruthGPT 
5 -- Reddit API要對開發AI模型獲利的用戶收費 
6 -- 以AI生成的黑白照片獲索尼世界攝影大賽首獎，創作者拒絕領獎 
7 -- 外傳三星正考慮以Bing取代Google作為裝置預設搜尋引擎 
8 -- 傳馬斯克已成立開發ChatGPT競爭服務的AI公司 
9 -- Google訓練機器人在自家辦公室分類垃圾 
10 -- Meta開源可以把素描變成動畫的AI專案Animated Drawings 
Getting categorical news: Cloud
Getting categorical news url: https://www.ithome.com.tw/cloud
1 -- Pulumi整合大型語言模型，自動化雲端基礎設施即程式碼操作 
2 -- Visual Studio 17.6將加入建置加速平臺Incredibuild 10 
3 -- Amazon推出雲端AI模型平臺Bedrock、大型語言模型Titan 
4 -- ASP.NET Core鎖定雲端應用程式提供原生AOT支援，程式啟動更快記憶體消耗更少 
5 -- Wasmer發表WCGI讓開發者將CGI程式編譯成WebAssembly 
6 -- 資安公司監測160萬人，竟有3.1%員工上傳機敏資料到ChatGPT 
7 -- AWS Service Catalog讓用戶也能使用Terraform管控雲端資源 
8 -- Nvidia雲端新戰略 
9 -- 【ChatGPT浪潮來襲，帶動平價客製化AI模型新需求】Nvidia大進擊，從AI軟硬體商變成雲端AI供應商 
10 -- Databricks針對製造業推出資料湖倉，加速資料分析和AI用例開發 


In [6]:
# page
# print(items)
# dtstr
# content


In [7]:
import pandas as pd

data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [8]:
df.head(2)

,item_id,date,category,title,content,link,photo_link
0,big-data_20230420_1,2023-04-20,AI,微軟傳已投入開發自有AI專用晶片,《The Information》報導，積極投入AI策略的微軟，數年前已著手開發一顆專門為...,https://www.ithome.com.tw//news/156484,https://s4.itho.me/sites/default/files/styles/...
1,big-data_20230420_2,2023-04-20,AI,Adobe Lightroom新增以AI消除噪點、強化細節修整的功能,本周在NAB產品大會上，Adobe宣布多項整合AI，可消除相片噪點、調整圖片細節的新Lig...,https://www.ithome.com.tw//news/156482,https://s4.itho.me/sites/default/files/styles/...


In [9]:
df.to_csv("ithome_categories_news.csv", sep="|", index=False)

In [10]:
df = pd.read_csv("ithome_categories_news.csv", sep="|")
# df.head(2)
print("1.", df.content[0])
print("2.", df.shape)

1.  《The Information》報導，積極投入AI策略的微軟，數年前已著手開發一顆專門為AI模型訓練及執行聊天機器人而設計的強大晶片，代號為Athena。
報導引述2名消息人士指出，微軟早在2019年就已啟動Athena專案的開發，希望這顆軟、硬整合設計的晶片能提供比市售晶片更大的運算效能。微軟2019年首次投資AI新創公司OpenAI 10億美元。消息人士也說，已有部分微軟和OpenAI員工已經拿到Athena的工程樣品。
報導指出，目前有300人投入Athena專案開發。Athena專案規畫數代設計，第一代可能以台積電5奈米製程生產。
微軟最近投資OpenAI數十億美元，取得其GPT模型及ChatGPT技術。微軟最終目的是希望將AI整合到它所有產品服務中，包括Office、Windows、甚至整個Azure雲端服務。而一旦以高效能晶片作為Azure雲端的運行底層，將需天價硬體成本。
專家分析，現有AI系統使用的GPU中，Nvidia的A100及H100占了90% 的市場，兩顆售價分別在每顆1萬及4萬美元。透過自行研發晶片，微軟可能希望藉此降低資料中心成本，提升開發主導性及開發時程，或至少取得和Nvidia議價的籌碼。
微軟和Nvidia皆未對此回應。
這並不是微軟第一次投入AI硬體開發。微軟2017年也透過Brainwave專案，開發在雲端及邊緣執行即時AI推論的深度學習平臺。
其他的雲端大廠，包括Google、Amazon Web Services、騰訊、阿里巴巴等也都投入自製AI晶片。

2. (20, 7)


In [11]:
# df.head()
df.content[0:5]

0     《The Information》報導，積極投入AI策略的微軟，數年前已著手開發一顆專門為...
1     本周在NAB產品大會上，Adobe宣布多項整合AI，可消除相片噪點、調整圖片細節的新Lig...
2     Snapchat於今年2月已經宣布與OpenAI合作推出名為「我的 AI（My AI）」的...
3     推特及特斯拉執行長馬斯克（Elon Musk）本周表示要開發不說謊，更安全的AI聊天機器人...
4     有鑒於OpenAI或其他AI業者利用Reddit上的程式碼等資源，開源碼程式碼代管平臺Re...
Name: content, dtype: object

In [12]:
import numpy

In [13]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


### **WS 斷詞**

In [14]:
%%time
# It takes time.
tokens=ws(df.content)

Inference: 100%|██████████| 1/1 [00:15<00:00, 15.77s/it]

Wall time: 16.1 s


### **POS詞性標記**

In [15]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 5/5 [00:44<00:00,  8.81s/it]

Wall time: 44.3 s


In [16]:
print("1.",len(tokens_pos))
print("2.",len(tokens_pos[1]))
print("3.",tokens_pos[0])

1. 20
2. 412
3. ['WHITESPACE', 'PARENTHESISCATEGORY', 'FW', 'WHITESPACE', 'FW', 'PARENTHESISCATEGORY', 'VE', 'COMMACATEGORY', 'VH', 'VC', 'FW', 'Na', 'DE', 'Nb', 'COMMACATEGORY', 'Neu', 'Nf', 'Ng', 'D', 'VF', 'VC', 'Neu', 'Nf', 'VL', 'P', 'FW', 'Na', 'Na', 'Caa', 'VC', 'VA', 'Na', 'Cbb', 'VC', 'DE', 'VH', 'Na', 'COMMACATEGORY', 'Na', 'VG', 'FW', 'PERIODCATEGORY', 'WHITESPACE', 'Na', 'VC', 'Neu', 'Nf', 'Na', 'Na', 'VE', 'COMMACATEGORY', 'Nb', 'D', 'P', 'Neu', 'D', 'D', 'VC', 'FW', 'Na', 'DE', 'Nv', 'COMMACATEGORY', 'VK', 'Nep', 'Nf', 'VH', 'PAUSECATEGORY', 'VC', 'Na', 'DE', 'Na', 'D', 'VD', 'P', 'A', 'Na', 'Dfa', 'VH', 'DE', 'Nv', 'Na', 'PERIODCATEGORY', 'Nb', 'Neu', 'D', 'VC', 'FW', 'VC', 'Nc', 'FW', 'Nf', 'PERIODCATEGORY', 'Na', 'Na', 'D', 'VE', 'COMMACATEGORY', 'D', 'V_2', 'Neqa', 'Nb', 'Caa', 'FW', 'Na', 'D', 'VC', 'FW', 'DE', 'Na', 'Na', 'PERIODCATEGORY', 'WHITESPACE', 'VE', 'VE', 'COMMACATEGORY', 'Nd', 'V_2', 'Neu', 'Na', 'VC', 'FW', 'Na', 'VC', 'PERIODCATEGORY', 'FW', 'Na', 'Na',

### **NER 實體辨識**

In [17]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 1/1 [00:16<00:00, 16.93s/it]


Wall time: 17.3 s


In [18]:
entity_list[0]

[NerToken(word='微軟', ner='ORG', idx=(30, 32)),
 NerToken(word='數年前', ner='DATE', idx=(33, 36)),
 NerToken(word='2019年', ner='DATE', idx=(97, 102)),
 NerToken(word='Athena', ner='ORG', idx=(106, 112)),
 NerToken(word='微軟', ner='ORG', idx=(148, 150)),
 NerToken(word='微軟', ner='ORG', idx=(189, 191)),
 NerToken(word='Athena', ner='ORG', idx=(204, 210)),
 NerToken(word='Athena', ner='ORG', idx=(231, 237)),
 NerToken(word='台積電5奈米', ner='ORG', idx=(263, 269)),
 NerToken(word='微軟', ner='ORG', idx=(275, 277)),
 NerToken(word='ChatGPT', ner='ORG', idx=(302, 309)),
 NerToken(word='微軟', ner='ORG', idx=(312, 314)),
 NerToken(word='AI', ner='ORG', idx=(322, 324)),
 NerToken(word='Windows', ner='ORG', idx=(345, 352)),
 NerToken(word='GPU', ner='ORG', idx=(415, 418)),
 NerToken(word='Nvidia', ner='ORG', idx=(420, 426)),
 NerToken(word='4萬美元', ner='MONEY', idx=(458, 462)),
 NerToken(word='idia', ner='ORG', idx=(510, 514)),
 NerToken(word='微軟', ner='ORG', idx=(521, 523)),
 NerToken(word='Nvidia', ner='O

In [19]:
word_pos_pair = [list(zip(w,p)) for w, p in zip(tokens, tokens_pos)]

print("1.", len(word_pos_pair))
print("2.", word_pos_pair[1][0])

1. 20
2. (' ', 'WHITESPACE')


In [20]:
# with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#     stops = f.read().split('\n') 

In [21]:
allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

tokens_v2[0]

['投入',
 '策略',
 '微軟',
 '開發',
 '模型',
 '訓練',
 '執行',
 '聊天',
 '機器人',
 '設計',
 '晶片',
 '代號',
 '報導',
 '引述',
 '消息',
 '人士',
 '微軟',
 '啟動',
 '專案',
 '整合',
 '設計',
 '晶片',
 '晶片',
 '效能',
 '微軟',
 '投資',
 '新創',
 '公司',
 '消息',
 '人士',
 '微軟',
 '員工',
 '拿到',
 '工程',
 '樣品',
 '投入',
 '專案',
 '開發',
 '專案',
 '規畫',
 '設計',
 '台積電',
 '奈米',
 '生產',
 '微軟',
 '投資',
 '取得',
 '模型',
 '技術',
 '微軟',
 '目的',
 '整合到',
 '產品',
 '服務',
 '雲端',
 '服務',
 '高效能',
 '晶片',
 '雲端',
 '運行',
 '底層',
 '天價硬體',
 '成本',
 '分析',
 '系統',
 '使用',
 '市場',
 '售價',
 '研發',
 '晶片',
 '微軟',
 '資料',
 '中心',
 '成本',
 '提升',
 '開發',
 '主導性',
 '開發',
 '時程',
 '取得和',
 '議價',
 '籌碼',
 '微軟',
 '回應',
 '微軟',
 '投入',
 '硬體',
 '開發',
 '微軟',
 '專案',
 '開發',
 '雲端',
 '邊緣',
 '執行',
 '推論',
 '深度',
 '平臺',
 '雲端',
 '騰訊',
 '阿里巴巴',
 '投入',
 '自製',
 '晶片']

### **新增段詞資料欄位**

In [22]:
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

In [23]:
# df

### **統計字頻**

In [24]:
# allowPOS過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']

In [25]:
from collections import Counter
def word_frequency(word_pos_pair):
    filtered_words =[]
    for word, pos in word_pos_pair:
        if (pos in allowPOS) & (len(word) >= 2):            
            filtered_words.append(word)
        #print('%s %s' % (word, flag))
    c = Counter( filtered_words)
    return c.most_common(30)

# 看過濾後重新搭配的: word_pos_pair["第幾篇"]

In [26]:
words = word_frequency(word_pos_pair[1])
words

[('功能', 9),
 ('編輯', 6),
 ('影像', 5),
 ('加入', 4),
 ('人像', 4),
 ('用戶', 4),
 ('支援', 4),
 ('相片', 3),
 ('細節', 3),
 ('顏色', 3),
 ('檔案', 3),
 ('色調', 3),
 ('影片', 3),
 ('宣布', 2),
 ('噪點', 2),
 ('調整', 2),
 ('圖片', 2),
 ('軟體', 2),
 ('行動版', 2),
 ('模型', 2),
 ('鬍鬚', 2),
 ('處理', 2),
 ('曲線', 2),
 ('內容', 2),
 ('真實性', 2),
 ('文字', 2),
 ('來源', 2),
 ('產品', 1),
 ('大會', 1),
 ('整合', 1)]

In [27]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)  

# keyfreqs[0]: 看第0篇過濾後斷詞的字頻
# keyfreqs[0:2]: 看第0篇與第1篇過濾後斷詞的字頻

# 再次新增欄位到資料集
df['top_key_freq'] = keyfreqs
df.head(3)

Wall time: 13.9 ms


,item_id,date,category,title,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,big-data_20230420_1,2023-04-20,AI,微軟傳已投入開發自有AI專用晶片,《The Information》報導，積極投入AI策略的微軟，數年前已著手開發一顆專門為...,https://www.ithome.com.tw//news/156484,https://s4.itho.me/sites/default/files/styles/...,"[ , 《, The, , Information, 》, 報導, ，, 積極, 投入, ...","[投入, 策略, 微軟, 開發, 模型, 訓練, 執行, 聊天, 機器人, 設計, 晶片, ...","[(微軟, ORG, (30, 32)), (數年前, DATE, (33, 36)), (...","[( , WHITESPACE), (《, PARENTHESISCATEGORY), (T...","[(微軟, 10), (開發, 6), (晶片, 6), (投入, 4), (專案, 4),..."
1,big-data_20230420_2,2023-04-20,AI,Adobe Lightroom新增以AI消除噪點、強化細節修整的功能,本周在NAB產品大會上，Adobe宣布多項整合AI，可消除相片噪點、調整圖片細節的新Lig...,https://www.ithome.com.tw//news/156482,https://s4.itho.me/sites/default/files/styles/...,"[ , 本, 周, 在, NAB, 產品, 大會, 上, ，, Adobe, 宣布, 多, ...","[產品, 大會, 宣布, 整合, 消除, 相片, 噪點, 調整, 圖片, 細節, 功能, 相...","[(本周, DATE, (1, 3)), (Adobe, PERSON, (13, 18))...","[( , WHITESPACE), (本, Nes), (周, Nd), (在, P), (...","[(功能, 9), (編輯, 6), (影像, 5), (加入, 4), (人像, 4), ..."
2,big-data_20230420_3,2023-04-20,AI,Snapchat開放聊天機器人供全球用戶免費體驗，推出AI生成的濾鏡,Snapchat於今年2月已經宣布與OpenAI合作推出名為「我的 AI（My AI）」的...,https://www.ithome.com.tw//news/156481,https://s4.itho.me/sites/default/files/styles/...,"[ Snapchat, 於, 今年, 2月, 已經, 宣布, 與, Open, AI, 合作...","[宣布, 推出, 智慧, 助理, 付費, 訂閱, 用戶, 美金, 世界, 使用, 聊天, 機...","[(今年, DATE, (10, 12)), (2月, DATE, (12, 14)), (...","[( Snapchat, FW), (於, P), (今年, Nd), (2月, Nd), ...","[(濾鏡, 11), (用戶, 9), (機器人, 5), (使用, 4), (聊天, 4)..."


In [28]:
df.columns

Index(['item_id', 'date', 'category', 'title', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq'],
      dtype='object')

In [29]:
df=df[['item_id', 'date', 'category', 'title', 'content', 'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link']]
df.head(1)

,item_id,date,category,title,content,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,big-data_20230420_1,2023-04-20,AI,微軟傳已投入開發自有AI專用晶片,《The Information》報導，積極投入AI策略的微軟，數年前已著手開發一顆專門為...,"[(微軟, 10), (開發, 6), (晶片, 6), (投入, 4), (專案, 4),...","[ , 《, The, , Information, 》, 報導, ，, 積極, 投入, ...","[投入, 策略, 微軟, 開發, 模型, 訓練, 執行, 聊天, 機器人, 設計, 晶片, ...","[(微軟, ORG, (30, 32)), (數年前, DATE, (33, 36)), (...","[( , WHITESPACE), (《, PARENTHESISCATEGORY), (T...",https://www.ithome.com.tw//news/156484,https://s4.itho.me/sites/default/files/styles/...


In [30]:
df.to_csv('ithome_dataset_processed.csv', sep='|', index=False)

In [31]:
df = pd.read_csv('ithome_dataset_processed.csv', sep='|')
df.head(3)

,item_id,date,category,title,content,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,big-data_20230420_1,2023-04-20,AI,微軟傳已投入開發自有AI專用晶片,《The Information》報導，積極投入AI策略的微軟，數年前已著手開發一顆專門為...,"[('微軟', 10), ('開發', 6), ('晶片', 6), ('投入', 4), ...","[' ', '《', 'The', ' ', 'Information', '》', '報導...","['投入', '策略', '微軟', '開發', '模型', '訓練', '執行', '聊天...","[NerToken(word='微軟', ner='ORG', idx=(30, 32)),...","[(' ', 'WHITESPACE'), ('《', 'PARENTHESISCATEGO...",https://www.ithome.com.tw//news/156484,https://s4.itho.me/sites/default/files/styles/...
1,big-data_20230420_2,2023-04-20,AI,Adobe Lightroom新增以AI消除噪點、強化細節修整的功能,本周在NAB產品大會上，Adobe宣布多項整合AI，可消除相片噪點、調整圖片細節的新Lig...,"[('功能', 9), ('編輯', 6), ('影像', 5), ('加入', 4), (...","[' ', '本', '周', '在', 'NAB', '產品', '大會', '上', '...","['產品', '大會', '宣布', '整合', '消除', '相片', '噪點', '調整...","[NerToken(word='本周', ner='DATE', idx=(1, 3)), ...","[(' ', 'WHITESPACE'), ('本', 'Nes'), ('周', 'Nd'...",https://www.ithome.com.tw//news/156482,https://s4.itho.me/sites/default/files/styles/...
2,big-data_20230420_3,2023-04-20,AI,Snapchat開放聊天機器人供全球用戶免費體驗，推出AI生成的濾鏡,Snapchat於今年2月已經宣布與OpenAI合作推出名為「我的 AI（My AI）」的...,"[('濾鏡', 11), ('用戶', 9), ('機器人', 5), ('使用', 4),...","[' Snapchat', '於', '今年', '2月', '已經', '宣布', '與'...","['宣布', '推出', '智慧', '助理', '付費', '訂閱', '用戶', '美金...","[NerToken(word='今年', ner='DATE', idx=(10, 12))...","[(' Snapchat', 'FW'), ('於', 'P'), ('今年', 'Nd')...",https://www.ithome.com.tw//news/156481,https://s4.itho.me/sites/default/files/styles/...


In [32]:
# Create a new DataFrame with only the 'title' and 'summary' columns
# df_summary = df[['category','title', 'summary']]

# df_summary.to_csv('ithome_summary_with_title.csv', index=False)


In [33]:
from collections import Counter

# 再次過濾: 兩個字以上、特定的詞性
# Filter condition: two words and specified POS
allowedPOS=['Na','Nb','Nc']

In [34]:
# 使用token_pos
def get_top_words():
        top_cate_words={} # final result
        counter_all = Counter() # counter for category '全部'
        for category in news_categories:
            
            # df yesterday-->This is your homework
            # df=df[(df.date >= start_date) & (df.date <= end_date)]
            
            # df category
            df_group = df[df.category == category]
            
            # concatenate all filtered words in the same category
            words_group = []
            for row in df_group.token_pos:
                
                # filter words for each news
                filtered_words =[]
                for (word, pos) in eval(row):
                    if (len(word) >= 2) & (pos in allowedPOS):
                        filtered_words.append(word)
                        
                # concatenate filtered words  
                words_group += filtered_words

            # now we can count word frequency
            counter = Counter( words_group )
            
            # counter 
            counter_all += counter
            topwords = counter.most_common(30)

            # store topwords
            top_cate_words[category]= topwords

        # Process category '全部'
        top_cate_words['全部'] = counter_all.most_common(30)

        return list(top_cate_words.items())

In [35]:
%%time
result = get_top_words()

result[0:2]

Wall time: 153 ms


[('AI',
  [('機器人', 29),
   ('用戶', 16),
   ('馬斯克', 15),
   ('垃圾', 15),
   ('素描', 15),
   ('功能', 14),
   ('微軟', 13),
   ('分類', 13),
   ('模型', 12),
   ('專案', 11),
   ('濾鏡', 11),
   ('人員', 11),
   ('平臺', 10),
   ('圖片', 10),
   ('公司', 9),
   ('資料', 9),
   ('作品', 9),
   ('照片', 8),
   ('內容', 7),
   ('人類', 7),
   ('晶片', 6),
   ('技術', 6),
   ('影像', 6),
   ('世界', 6),
   ('工具', 6),
   ('攝影', 6),
   ('經驗', 6),
   ('動畫', 6),
   ('目的', 5),
   ('產品', 5)]),
 ('Cloud',
  [('資料', 82),
   ('模型', 65),
   ('企業', 48),
   ('程式', 41),
   ('雲端', 38),
   ('基礎', 30),
   ('應用', 30),
   ('電腦', 28),
   ('用戶', 22),
   ('資源', 21),
   ('內容', 19),
   ('軟體', 18),
   ('架構', 18),
   ('工具', 17),
   ('員工', 17),
   ('設施', 14),
   ('功能', 14),
   ('技術', 13),
   ('程式碼', 12),
   ('工作', 11),
   ('文字', 11),
   ('訓練', 11),
   ('伺服器', 11),
   ('時間', 10),
   ('生成式', 9),
   ('語言', 9),
   ('任務', 9),
   ('平臺', 9),
   ('微軟', 9),
   ('參數', 9)])]

In [36]:
row0 = df.iloc[0].token_pos
row0
# type(row0) 是 str

"[(' ', 'WHITESPACE'), ('《', 'PARENTHESISCATEGORY'), ('The', 'FW'), (' ', 'WHITESPACE'), ('Information', 'FW'), ('》', 'PARENTHESISCATEGORY'), ('報導', 'VE'), ('，', 'COMMACATEGORY'), ('積極', 'VH'), ('投入', 'VC'), ('AI', 'FW'), ('策略', 'Na'), ('的', 'DE'), ('微軟', 'Nb'), ('，', 'COMMACATEGORY'), ('數', 'Neu'), ('年', 'Nf'), ('前', 'Ng'), ('已', 'D'), ('著手', 'VF'), ('開發', 'VC'), ('一', 'Neu'), ('顆', 'Nf'), ('專門', 'VL'), ('為', 'P'), ('AI', 'FW'), ('模型', 'Na'), ('訓練', 'Na'), ('及', 'Caa'), ('執行', 'VC'), ('聊天', 'VA'), ('機器人', 'Na'), ('而', 'Cbb'), ('設計', 'VC'), ('的', 'DE'), ('強大', 'VH'), ('晶片', 'Na'), ('，', 'COMMACATEGORY'), ('代號', 'Na'), ('為', 'VG'), ('Athena', 'FW'), ('。', 'PERIODCATEGORY'), ('\\n', 'WHITESPACE'), ('報導', 'Na'), ('引述', 'VC'), ('2', 'Neu'), ('名', 'Nf'), ('消息', 'Na'), ('人士', 'Na'), ('指出', 'VE'), ('，', 'COMMACATEGORY'), ('微軟', 'Nb'), ('早', 'D'), ('在', 'P'), ('2019年', 'Neu'), ('就', 'D'), ('已', 'D'), ('啟動', 'VC'), ('Athena', 'FW'), ('專案', 'Na'), ('的', 'DE'), ('開發', 'Nv'), ('，', 'COMMACATEGORY'

In [37]:
# eval: 不論是整數或浮點數都會自動轉型成可以做計算的數值
row0 = eval(df.iloc[0].token_pos)
row1 = eval(df.iloc[1].token_pos)
row01 = row0+row1

In [38]:
c0 =Counter( row0 )
c1 =Counter( row1 )

c0.most_common(10)
c1.most_common(10)

c01 = c0+c1
c01.most_common(10)

[(('，', 'COMMACATEGORY'), 48),
 (('的', 'DE'), 29),
 (('。', 'PERIODCATEGORY'), 27),
 (('AI', 'FW'), 15),
 (('及', 'Caa'), 12),
 (('、', 'PAUSECATEGORY'), 12),
 (('微軟', 'Nb'), 10),
 (('\n', 'WHITESPACE'), 9),
 (('可', 'D'), 9),
 (('功能', 'Na'), 9)]

In [39]:
top_group_words = get_top_words()
df_top_group_words = pd.DataFrame(top_group_words, columns = ['category','top_keys'])
df_top_group_words

,category,top_keys
0,AI,"[(機器人, 29), (用戶, 16), (馬斯克, 15), (垃圾, 15), (素描..."
1,Cloud,"[(資料, 82), (模型, 65), (企業, 48), (程式, 41), (雲端, ..."
2,全部,"[(資料, 91), (模型, 77), (企業, 49), (程式, 43), (雲端, ..."


In [40]:
df_top_group_words.to_csv('ithome_news_topkey_with_category_via_token_pos.csv', index=False)

In [41]:
df_data = pd.read_csv('ithome_news_topkey_with_category_via_token_pos.csv',sep=',')
# df_data.head(12)

# df_data['top_keys'][0] 為字串，並非list
eval(df_data['top_keys'][0])

[('機器人', 29),
 ('用戶', 16),
 ('馬斯克', 15),
 ('垃圾', 15),
 ('素描', 15),
 ('功能', 14),
 ('微軟', 13),
 ('分類', 13),
 ('模型', 12),
 ('專案', 11),
 ('濾鏡', 11),
 ('人員', 11),
 ('平臺', 10),
 ('圖片', 10),
 ('公司', 9),
 ('資料', 9),
 ('作品', 9),
 ('照片', 8),
 ('內容', 7),
 ('人類', 7),
 ('晶片', 6),
 ('技術', 6),
 ('影像', 6),
 ('世界', 6),
 ('工具', 6),
 ('攝影', 6),
 ('經驗', 6),
 ('動畫', 6),
 ('目的', 5),
 ('產品', 5)]

In [42]:
data={}
for idx, row in df_data.iterrows():
    data[row['category']]= eval(row['top_keys'])

# data
data['AI'][0:3]

[('機器人', 29), ('用戶', 16), ('馬斯克', 15)]

In [43]:
# sum={}
# for idx, row in df_summary.iterrows():
#     sum[row['title']]= eval(row['summary'])
# sum